In [1]:
from PIL import Image, ImageOps
import numpy as np
from numpy import asarray, zeros, empty, sign, array_str, flip, int64
import math
from operator import xor
import pandas as pd
import random

In [2]:
print("-------- ШАГ 1 --------")
IMG = Image.open("test1.bmp") #открываем изображение
W = ImageOps.grayscale(Image.open("key-CVZ.bmp")) #открываем ч/б ЦВЗ
print(f"Формат контейнера: {IMG.format}")
print(f"Размеры контейнера: {IMG.size}")
print(f"Режим контейнера: {IMG.mode}")

X,Y = IMG.size #присваивание размеров контейнера переменным X и Y

rValues,gValues,bValues = IMG.split() #Разделение контейнера на три канала

bValues = asarray(bValues) #переводим синий канал в массив пикселей
print(f"\nblue channel =\n{bValues}")

-------- ШАГ 1 --------
Формат контейнера: BMP
Размеры контейнера: (128, 128)
Режим контейнера: RGB

blue channel =
[[170 171 171 ... 203 203 204]
 [171 172 172 ... 204 203 204]
 [171 171 172 ... 204 204 204]
 ...
 [ 94  95  94 ... 106 106 109]
 [ 91  92  92 ... 107 109 111]
 [ 88  89  90 ... 109 110 112]]


In [3]:
print("-------- ШАГ 2 --------")
F = zeros(shape=(X,Y))
def FF():
    d = -1
    for x in range(0, X):
        d = -d
        for y in range(0, Y):
            F[x][y] = d
            d = -d
    return F
print(f"F =\n{FF()}")

-------- ШАГ 2 --------
F =
[[ 1. -1.  1. ... -1.  1. -1.]
 [-1.  1. -1. ...  1. -1.  1.]
 [ 1. -1.  1. ... -1.  1. -1.]
 ...
 [-1.  1. -1. ...  1. -1.  1.]
 [ 1. -1.  1. ... -1.  1. -1.]
 [-1.  1. -1. ...  1. -1.  1.]]


In [4]:
print("-------- ШАГ 3 --------")

Nf = 1024
n = math.floor(math.sqrt((X*Y)/Nf))
print(f"Nf = {Nf}, n = {n}")

-------- ШАГ 3 --------
Nf = 1024, n = 4


In [5]:
print("-------- ШАГ 4 --------")

d = int(math.log(Nf,2))
s = 55
print(f"d = {d}, s = {s}")

def B2D(x):
    sum = 0
    for i in range(10):
        if x[i] == 1:
            sum += pow(2,i)
    return sum

def D2B(x):
    V = empty(10, dtype=int)
    znak = sign(x)
    for i in range(10):
        V[i] = abs(x) % 2
        x = math.floor(abs(x)/2)
    return znak*V

def Vmd(s):
    mu = [1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
    mu = asarray(mu)
    mu = mu.reshape((-1, 1))
    Rdec = empty(pow(2,d), dtype=int)
    Rbin = empty(d, dtype=int)
    bitar = zeros(d, dtype=int)

    for i in range(0, pow(2,d)-1):
        if i == 0:
            Rdec[i] = s
            Rbin = D2B(Rdec[i])
        elif i >= 1:
            bit = 0
            k = 0
            for j in range(0, d):
                if mu[j] == 1:
                    bit = xor(Rbin[j], bit)
                    bitar[k] = bit
                    k += 1
            R = Rbin.copy()

            for j in range(0, d):
                if j == 0:
                    Rbin[j] = bit
                elif j > 0:
                    Rbin[j] = R[j-1]
            Rdec[i] = B2D(Rbin)
    Rdec[pow(2,d)-1] = pow(2,d)
    return Rdec


Vmd = Vmd(s)
print(f"Vmd(s) = {Vmd}")

-------- ШАГ 4 --------
d = 10, s = 55
Vmd(s) = [  55  111  222 ...  269  539 1024]


In [6]:
print("-------- ШАГ 5 --------")

def f():
    Ci = Vmd.copy()
    f = empty(pow(2, d), dtype=object)
    F0 = 0*F
    c1 = 0
    c2 = n
    for i in range(0, Nf):
        r1 = (n*(i)) % X
        r2 = r1+n
        Cik = Ci[i]-1
        F0[r1:r1+n, c1:c1+n] = F[r1:r2, c1:c2]
        f[Cik] = F0.astype(int64)
        if r2 == X:
            c1 = c1+n
            c2 = c2+n
        F0 = 0 * F
    return f
        

ff = f()
print(f"Сформирован массив базисных функций ff")

-------- ШАГ 5 --------
Сформирован массив базисных функций ff


In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pdff54 = pd.DataFrame(ff[54])
print(f"Базисная функция №55:\n{pdff54}")

Базисная функция №55:
     0    1    2    3    4    5    6    7    8    9    10   11   12   13   \
0    1   -1    1   -1    0    0    0    0    0    0    0    0    0    0     
1   -1    1   -1    1    0    0    0    0    0    0    0    0    0    0     
2    1   -1    1   -1    0    0    0    0    0    0    0    0    0    0     
3   -1    1   -1    1    0    0    0    0    0    0    0    0    0    0     
4    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
5    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
6    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
7    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
8    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
9    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
10   0    0    0    0    0    0    0    0    0    0    0    0    0    0     
11   0    0    0    0    0    0    0    0    0    0   

C:\Users\rushu\.conda\envs\pytorchgpu\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [10]:
print("-------- ШАГ 6 --------")

def delta():
    delta = zeros(pow(2, d), dtype=int)
    for i in range(Nf):
        for x in range(X):
            for y in range(Y):
                delta[i] += (ff[i])[x][y]*bValues[x][y]
    return delta
dlt = delta()
MaxAbsDlt = max(abs(dlt))
print(f"Максимальная абсолютная погрешность ортоганальности:\n{MaxAbsDlt}")

-------- ШАГ 6 --------
Максимальная абсолютная погрешность ортоганальности:
71


In [12]:
print("-------- ШАГ 7 --------")
print(f"Размер ЦВЗ: {W.size}\n")
W = np.array(W)
print(f"ЦВЗ:\n{W}\n")

def M():
    M = zeros(pow(2, d), dtype=int)
    k = 0
    for i in range(32):
        for j in range(32):
            M[k] = W[i,j]/255
            k += 1
    return M
M = M()
print(f"Одномерный массив M = {M}")

-------- ШАГ 7 --------
Размер ЦВЗ: 1024

ЦВЗ:
[[255 255 255 ... 255 255 255]
 [255   0   0 ...   0 255 255]
 [255   0 255 ...   0 255 255]
 ...
 [255   0   0 ...   0 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]

Одномерный массив M = [1 1 1 ... 1 1 1]


In [13]:
print("-------- ШАГ 8 --------")
Lm = 1024
def E():
    E = zeros((128, 128), dtype=int)
    Mvec = M
    Mvec_bin = (Mvec[0])
    for j in range(1, 1024):
        Mvec_bin = np.append(Mvec_bin, Mvec[j])
    for j in range(0, Lm):
        if Mvec_bin[j] == 0:
            Mvec_bin[j] = -1
    for x in range(X):
        for y in range(Y):
            for i in range(Lm):
                E[x][y] += Mvec_bin[i]*(ff[i])[x][y]
    return E

E = E()
print(f"\nМассив модулированного ЦВЗ E =\n{E}")

-------- ШАГ 8 --------

Массив модулированного ЦВЗ E =
[[ 1 -1  1 ... -1  1 -1]
 [-1  1 -1 ...  1 -1  1]
 [ 1 -1  1 ... -1  1 -1]
 ...
 [-1  1 -1 ...  1 -1  1]
 [ 1 -1  1 ... -1  1 -1]
 [-1  1 -1 ...  1 -1  1]]


In [14]:
print("-------- ШАГ 9 --------")

def Find():
    Kg = 1
    nff = pow(n, 2)
    a = 1
    while a == 1:
        if nff*Kg > MaxAbsDlt:
            return Kg
        else:
            Kg += 1

Kg = Find()
print(f"\nДостаточный коэффициент усиления по мощности Kg = {Kg}")

-------- ШАГ 9 --------

Достаточный коэффициент усиления по мощности Kg = 5


In [15]:
print("-------- ШАГ 10 --------")

def Cnorm():
    Cnorm = zeros((X, Y), dtype=int)
    for x in range(X):
        for y in range(Y):
            Cnorm[x][y] = round((bValues[x][y]/255)*(255-2*Kg)+Kg)
    return Cnorm
Cnorm = Cnorm()
print(f"\nmin(Cnorm) = {np.amin(Cnorm)}")
print(f"max(Cnorm) = {np.amax(Cnorm)}")
S = zeros((X, Y), dtype=int)
S = Cnorm + Kg*E
print(f"min(S) = {np.amin(S)}")
print(f"max(S) = {np.amax(S)}")
print(f"\nКонтейнер S=\n{S}")

-------- ШАГ 10 --------

min(Cnorm) = 42
max(Cnorm) = 202
min(S) = 37
max(S) = 207

Контейнер S=
[[173 164 174 ... 195 205 196]
 [164 175 165 ... 206 195 206]
 [174 164 175 ... 196 206 196]
 ...
 [ 90 101  90 ... 112 102 115]
 [ 97  88  98 ... 103 115 107]
 [ 85  96  86 ... 115 106 118]]


In [16]:
print("-------- ШАГ 11 --------")

def Mm():
    mu = 0
    Mvec_bin = empty(pow(2, d), dtype=int)
    for i in range(0, Nf):
        for x in range(X):
            for y in range(Y):
                fff = ff[i]
                mu += S[x][y]*fff[x][y]
        if mu > 0:
            Mvec_bin[i] = 1
        if mu < 0:
            Mvec_bin[i] = 0
        if mu == 0:
            Mvec_bin[i] = round(random.random())
        mu = 0
    return Mvec_bin

Mm = Mm()
print(f"\nОдномерный массив извлеченного ЦВЗ\nMm = {Mm}")

def W():
    W = zeros((32, 32), dtype=int)
    k = 0
    for i in range(32):
        for j in range(32):
            W[i][j] = Mm[k]*255
            k = k + 1
    return W

W = W()
print(f"\nИзвлеченный ЦВЗ:\n{W}")

-------- ШАГ 11 --------

Одномерный массив извлеченного ЦВЗ
Mm = [1 1 1 ... 1 1 1]

Извлеченный ЦВЗ:
[[255 255 255 ... 255 255 255]
 [255   0   0 ...   0 255 255]
 [255   0 255 ...   0 255 255]
 ...
 [255   0   0 ...   0 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [17]:
print("-------- МЕТРИКИ --------")
def MSE():
    MSE = 0
    for x in range(X):
        for y in range(Y):
            MSE += pow((bValues[x][y]-S[x][y]), 2)
    MSE = (1/(X*Y))*MSE
    return MSE

def NMSE():
    NMSE = 0
    UNMSE, DNMSE = 0, 0
    for x in range(X):
        for y in range(Y):
            UNMSE += pow((bValues[x][y]-S[x][y]), 2)
    for x in range(X):
        for y in range(Y):
            DNMSE += pow(bValues[x][y], 2)
    NMSE = UNMSE / DNMSE
    return NMSE

MSE = MSE()
NMSE = NMSE()
print(f"\nMSE = {MSE:.3f}")
print(f"\nNMSE = {NMSE:.3f}")

-------- МЕТРИКИ --------

MSE = 27.871

NMSE = 0.001


In [18]:
print("-------- ВЫГРУЗКА ИЗОБРАЖЕНИЙ --------")
#Вывод контейнера
S = Image.fromarray(S) #перевод из массива в формат изображения
S = S.convert('L') #перевод формата в одноканальное изображение
new_IMG = Image.merge('RGB', (rValues, gValues, S)) #объединение трех каналов
new_IMG.save("new_IMG.bmp") #Сохранение изображения в файле new_IMG.bmp
print("\nИзмененный контейнер сохранен в файле new_IMG.bmp")
#Вывод ЦВЗ
W = Image.fromarray(W)
W = W.convert('L')
W.save("new_W.bmp")
print("\nИзвлеченный ЦВЗ сохранен в файле new_W.bmp")

-------- ВЫГРУЗКА ИЗОБРАЖЕНИЙ --------

Измененный контейнер сохранен в файле new_IMG.bmp

Извлеченный ЦВЗ сохранен в файле new_W.bmp
